In [1]:
from pyspark.sql import SparkSession,Window
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json

In [3]:
!pip install ipykernel -U --user --force-reinstall

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.1/117.1 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 18.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.3/817.3 kB 77.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.9/105.9 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.9/919.9 kB 71.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 436.8/436.8 kB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.5/290.5 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.4/386.4 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!python --version

Python 3.11.9


In [5]:
spark=SparkSession.builder.\
    appName("Test 4").\
        getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/26 15:23:01 INFO SparkEnv: Registering MapOutputTracker
24/06/26 15:23:01 INFO SparkEnv: Registering BlockManagerMaster
24/06/26 15:23:01 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/06/26 15:23:01 INFO SparkEnv: Registering OutputCommitCoordinator


In [3]:
data=[(1,5),(2,6),(3,5),(3,6),(1,6)]
schema="customer_id int,product_key int"
customer_df=spark.createDataFrame(data,schema)


data=[(5,),(6,)]
schema="product_key int"
product_df=spark.createDataFrame(data,schema)

In [5]:
customer_df.show()
product_df.show()

+-----------+-----------+
|customer_id|product_key|
+-----------+-----------+
|          1|          5|
|          2|          6|
|          3|          5|
|          3|          6|
|          1|          6|
+-----------+-----------+

+-----------+
|product_key|
+-----------+
|          5|
|          6|
+-----------+



In [7]:
count_of_products=product_df.count()
    
count_of_products

2

In [9]:
customer_df_v2=customer_df.groupBy(col("customer_id")).\
    agg(countDistinct(col("product_key")).alias("count_of_distinct_product"))

In [10]:
customer_df_v2.show()

+-----------+-------------------------+
|customer_id|count_of_distinct_product|
+-----------+-------------------------+
|          1|                        2|
|          3|                        2|
|          2|                        1|
+-----------+-------------------------+



In [13]:
customer_df_v3=customer_df_v2.withColumn("all_products",when((col("count_of_distinct_product")==count_of_products),"true").\
    otherwise("false"))

In [14]:
customer_df_v3.show()

+-----------+-------------------------+------------+
|customer_id|count_of_distinct_product|all_products|
+-----------+-------------------------+------------+
|          1|                        2|        true|
|          3|                        2|        true|
|          2|                        1|       false|
+-----------+-------------------------+------------+



In [19]:
customer_df_v3.select(col("customer_id")).filter(col("all_products")=="true").show()

+-----------+
|customer_id|
+-----------+
|          1|
|          3|
+-----------+



In [22]:
data=[('Genece' , 2 , 75000),
('𝗝𝗮𝗶𝗺𝗶𝗻' , 2 , 80000 ),
('𝗣𝗮𝗻𝗸𝗮𝗷' , 2 , 80000 ),
('Tarvares' , 2 , 70000),
('Marlania' , 4 , 70000),
('Briana' , 4 , 85000),
('𝗞𝗶𝗺𝗯𝗲𝗿𝗹𝗶' , 4 , 55000),
('𝗚𝗮𝗯𝗿𝗶𝗲𝗹𝗹𝗮' , 4 , 55000),  
('Lakken', 5, 60000),
('Latoynia' , 5 , 65000) ]
schema="emp_name string,dept_id int,salary int"
df=spark.createDataFrame(data,schema)

In [23]:
df.show()

+------------------+-------+------+
|          emp_name|dept_id|salary|
+------------------+-------+------+
|            Genece|      2| 75000|
|      𝗝𝗮𝗶𝗺𝗶𝗻|      2| 80000|
|      𝗣𝗮𝗻𝗸𝗮𝗷|      2| 80000|
|          Tarvares|      2| 70000|
|          Marlania|      4| 70000|
|            Briana|      4| 85000|
|  𝗞𝗶𝗺𝗯𝗲𝗿𝗹𝗶|      4| 55000|
|𝗚𝗮𝗯𝗿𝗶𝗲𝗹𝗹𝗮|      4| 55000|
|            Lakken|      5| 60000|
|          Latoynia|      5| 65000|
+------------------+-------+------+



In [ ]:
#window_spec_max=Window.partitionBy(col("dept_id")).orderBy(col("salary"))

In [32]:
df_v2=df.\
    groupBy(col("dept_id")).\
    agg(max(col("salary")).alias("max_sal"),min(col("salary")).alias("min_sal"))
      

In [30]:
df_v2.show()

+-------+-------+-------+
|dept_id|max_sal|min_sal|
+-------+-------+-------+
|      4|  85000|  55000|
|      2|  80000|  70000|
|      5|  65000|  60000|
+-------+-------+-------+



In [40]:
df_alias = df.alias("df")
df_v2_alias = df_v2.alias("df_v2")

# Join, filter, and select columns
df_v3 = df_alias.join(df_v2_alias, df_alias["dept_id"] == df_v2_alias["dept_id"], how="left") \
    .filter((df_alias["salary"] == df_v2_alias["max_sal"]) | (df_alias["salary"] == df_v2_alias["min_sal"])) \
    .select(df_alias["emp_name"], df_alias["dept_id"])

In [41]:
df_v3.show()

+------------------+-------+
|          emp_name|dept_id|
+------------------+-------+
|      𝗝𝗮𝗶𝗺𝗶𝗻|      2|
|      𝗣𝗮𝗻𝗸𝗮𝗷|      2|
|          Tarvares|      2|
|            Briana|      4|
|  𝗞𝗶𝗺𝗯𝗲𝗿𝗹𝗶|      4|
|𝗚𝗮𝗯𝗿𝗶𝗲𝗹𝗹𝗮|      4|
|            Lakken|      5|
|          Latoynia|      5|
+------------------+-------+



In [15]:
df = spark.read.parquet("/user/sovik/nag_issues/20240618-095600797.parquet")
#df = spark.read.option("wholeFile", True).option("delimiter", ",").csv("/user/sovik/sample_lqs.csv",header=True)

In [16]:
df.show(truncate=False)

+------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------+-----------------+--------------------------------+-------------------+-------------------------------+------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [22]:
df.write.mode("overwrite").csv(path="/user/sovik/nag_issues/20240618-095600797.csv",header=True)

In [23]:
df_csv=spark.read.csv("/user/sovik/nag_issues/20240618-095600797.csv",header=True)

In [24]:
df_csv.show(truncate=False)

+------------------+-------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------+-----------------+--------------------------------+-----------------------+-------------------------------+------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [25]:
df_csv_doc=df_csv.select(col("_doc"))

In [32]:
df_csv_doc_json=df_csv_doc.toJSON()

In [33]:
json_value=df_csv_doc_json.collect()

In [34]:
json_value

['{"_doc":"{\\"_id\\":{\\"$oid\\":\\"655f0459c250e3e192b75533\\"},\\"likely_purchase_date\\":{\\"$date\\":\\"2023-11-30T00:00:00.000Z\\"},\\"status\\":\\"1668585610133\\",\\"current_followup_mode\\":\\"Call\\",\\"current_followup_mode_id\\":\\"1668406378088\\",\\"current_followup_stage\\":\\"Clarification about request\\",\\"current_followup_stage_id\\":\\"1665139043359\\",\\"email_reminder\\":1,\\"next_followup_mode\\":\\"Call\\",\\"next_followup_mode_id\\":\\"1668406378088\\",\\"next_followup_stage\\":\\"Clarification about request\\",\\"next_followup_stage_id\\":\\"1665139043359\\",\\"auto\\":0,\\"delete_event_data\\":1,\\"email_reminder_date\\":{\\"$date\\":\\"2023-11-23T13:45:00.000Z\\"},\\"lead_status\\":\\"Hot\\",\\"customer_email\\":\\"akash.singhavi@adglobal360.com\\",\\"outlet_id\\":\\"LQSDEALER1\\",\\"customer_name\\":\\"Akash\\",\\"lead_owner_id\\":\\"LQSDEALER1-LQSEMP1\\",\\"lead_lob\\":1,\\"followup_owner_id\\":\\"LQSDEALER1-LQSEMP1\\",\\"followup_owner_name\\":\\"LQSemp1

In [38]:
cleaned_data = [json.loads(json.loads(record)["_doc"].replace('\\"', '"')) for record in json_value]

In [44]:
df_new_json=spark.read.json(spark.sparkContext.parallelize([cleaned_data]))

In [45]:
df_new_json.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- auto: long (nullable = true)
 |-- client_id: long (nullable = true)
 |-- created_at: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- current_followup_mode: string (nullable = true)
 |-- current_followup_mode_id: string (nullable = true)
 |-- current_followup_stage: string (nullable = true)
 |-- current_followup_stage_id: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- delete_event_data: long (nullable = true)
 |-- email_reminder: long (nullable = true)
 |-- email_reminder_date: struct (nullable = true)
 |    |-- $date: string (nullable = true)
 |-- followup_owner_id: string (nullable = true)
 |-- followup_owner_name: string (nullable = true)
 |-- lead_id: struct (nullable = true)
 |    |-- $oid: string (nullable = true)
 |-- lead_lob: long (nullable = true)
 |-- lead_owner_id: string (nullable = true)
 |--

In [51]:
df_new_json.select(["_id.$oid"]).show()

+--------------------+
|                $oid|
+--------------------+
|655f0459c250e3e19...|
+--------------------+



In [52]:
df_1= spark.read.parquet("/user/sovik/nag_issues/20240621-114659524_bkp.parquet")

In [54]:
df_1.write.mode("overwrite").csv(path="/user/sovik/nag_issues/20240621-114659524_bkp.csv",header=True)

In [20]:
df_2= spark.read.parquet("/user/sovik/nag_issues/20240621-114686753.parquet")

In [21]:
df_2.show()

+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+----------------------+--------------------+--------------------+
|header__change_seq|header__change_oper| header__change_mask|header__stream_position|header__operation|header__transaction_id|  header__timestamp|header__partition_name|                 _id|                _doc|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+----------------------+--------------------+--------------------+
|           2.02E34|                  U|\x03\x00\x00\x00\...|   245;6385335149205...|           UPDATE|  4155544F434F4D4D4...|2024-06-07 10:04:50|  20240607T093000_2...|655f56e5be6c2c8cb...|{"_id":{"$oid":"6...|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+-------------------+------

In [22]:
df_2.write.mode("overwrite").csv(path="/user/sovik/nag_issues/20240621-114686753_v2.csv",header=True)

In [16]:
spark.read.csv(path="/user/sovik/nag_issues/20240621-114686753_v2.csv",header=True).show()

+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+--------------------+----------------------+--------------------+--------------------+
|header__change_seq|header__change_oper| header__change_mask|header__stream_position|header__operation|header__transaction_id|   header__timestamp|header__partition_name|                 _id|                _doc|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+--------------------+----------------------+--------------------+--------------------+
|           2.02E34|                  U|\x03\x00\x00\x00\...|   245;6385335149205...|           UPDATE|  4155544F434F4D4D4...|2024-06-07T10:04:...|  20240607T093000_2...|655f56e5be6c2c8cb...|{"_id":{"$oid":"6...|
+------------------+-------------------+--------------------+-----------------------+-----------------+----------------------+--------------------+-

In [6]:
help(spark.read)

Help on DataFrameReader in module pyspark.sql.readwriter object:

class DataFrameReader(OptionUtils)
 |  DataFrameReader(spark: 'SparkSession')
 |  
 |  Interface used to load a :class:`DataFrame` from external storage systems
 |  (e.g. file systems, key-value stores, etc). Use :attr:`SparkSession.read`
 |  to access this.
 |  
 |  .. versionadded:: 1.4.0
 |  
 |  .. versionchanged:: 3.4.0
 |      Supports Spark Connect.
 |  
 |  Method resolution order:
 |      DataFrameReader
 |      OptionUtils
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, spark: 'SparkSession')
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  csv(self, path: Union[str, List[str]], schema: Union[pyspark.sql.types.StructType, str, NoneType] = None, sep: Optional[str] = None, encoding: Optional[str] = None, quote: Optional[str] = None, escape: Optional[str] = None, comment: Optional[str] = None, header: Union[bool, str, NoneType] = None, inferSchema: Unio

In [72]:
df=spark.read.format("text").option("header",True).load(path="/user/sovik/nag_issues/sample_lqs_26062024.csv")

In [73]:
df.show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [75]:
df_list=df.collect()

In [76]:
df_list

[Row(value='"header__change_seq:STRING(35)","header__change_oper:STRING(1)","header__change_mask:BYTES(128)","header__stream_position:STRING(128)","header__operation:STRING(12)","header__transaction_id:STRING(32)","header__timestamp:DATETIME","_id:WSTRING(24)","_doc:NCLOB"'),
 Row(value='"20240626075915000000000000000000029","I",0180,"282;638549855603514910;20240212163654997880|667bca53:00000003:00000000","INSERT","4155544F434F4D4D4954000000000000",2024-06-26 07:59:15.000000,"667bca5365714e7849d07a2d","{""_id"": {""$oid"": ""667bca5365714e7849d07a2d""}, ""comments"": ""Followup Added"", ""email_reminder"": 1, ""email_reminder_date"": {""$date"": 1719408602000}, ""likely_purchase_date"": {""$date"": 1719964800000}, ""delete_event_data"": 1, ""auto"": 0, ""status"": ""1668585610133"", ""next_followup_mode"": ""Call"", ""next_followup_mode_id"": ""1668406378088"", ""followup_owner_id"": ""LQSDEALER1-007007"", ""followup_owner_name"": ""GP User"", ""lead_status"": ""Hot"", ""customer_email

In [77]:
df_header=df_list[0].value
df_value=df_list[1].value


In [78]:
df_header_v1=df_header.split(",")

In [79]:
df_header_v1

['"header__change_seq:STRING(35)"',
 '"header__change_oper:STRING(1)"',
 '"header__change_mask:BYTES(128)"',
 '"header__stream_position:STRING(128)"',
 '"header__operation:STRING(12)"',
 '"header__transaction_id:STRING(32)"',
 '"header__timestamp:DATETIME"',
 '"_id:WSTRING(24)"',
 '"_doc:NCLOB"']

In [80]:
df_header_doc=df_header_v1[-1]

In [81]:
df_header_doc

'"_doc:NCLOB"'

In [82]:
df_header_doc=df_header_doc[1:]

In [83]:
df_header_doc

'_doc:NCLOB"'

In [84]:
df_header_doc=df_header_doc[:-1]

In [85]:
df_header_doc

'_doc:NCLOB'

In [86]:
df_value

'"20240626075915000000000000000000029","I",0180,"282;638549855603514910;20240212163654997880|667bca53:00000003:00000000","INSERT","4155544F434F4D4D4954000000000000",2024-06-26 07:59:15.000000,"667bca5365714e7849d07a2d","{""_id"": {""$oid"": ""667bca5365714e7849d07a2d""}, ""comments"": ""Followup Added"", ""email_reminder"": 1, ""email_reminder_date"": {""$date"": 1719408602000}, ""likely_purchase_date"": {""$date"": 1719964800000}, ""delete_event_data"": 1, ""auto"": 0, ""status"": ""1668585610133"", ""next_followup_mode"": ""Call"", ""next_followup_mode_id"": ""1668406378088"", ""followup_owner_id"": ""LQSDEALER1-007007"", ""followup_owner_name"": ""GP User"", ""lead_status"": ""Hot"", ""customer_email"": ""akash.singhavi@adglobal360.com"", ""outlet_id"": ""LQSDEALER1"", ""customer_name"": ""Akash"", ""lead_owner_id"": ""LQSDEALER1-007007"", ""lead_lob"": 1, ""preferred_date_time"": ""2024-06-26T13:30:02.000Z"", ""created_at"": {""$date"": 1719408555000}, ""lead_id"": {""$oid"": ""667

In [87]:
startswith=df_value.find('{""_id""')

doc_value=df_value[startswith:]

type(doc_value)

str

In [98]:
doc_value=doc_value.replace('""','')


In [103]:
doc_value=doc_value[:-1]
doc_value

'{"_id": {"$oid": "667bca5365714e7849d07a2d"}, "comments": "Followup Added", "email_reminder": 1, "email_reminder_date": {"$date": 1719408602000}, "likely_purchase_date": {"$date": 1719964800000}, "delete_event_data": 1, "auto": 0, "status": "1668585610133", "next_followup_mode": "Call", "next_followup_mode_id": "1668406378088", "followup_owner_id": "LQSDEALER1-007007", "followup_owner_name": "GP User", "lead_status": "Hot", "customer_email": "akash.singhavi@adglobal360.com", "outlet_id": "LQSDEALER1", "customer_name": "Akash", "lead_owner_id": "LQSDEALER1-007007", "lead_lob": 1, "preferred_date_time": "2024-06-26T13:30:02.000Z", "created_at": {"$date": 1719408555000}, "lead_id": {"$oid": "667bca51b6292c3909ee4e46"}, "followup_added_by_distributor": 1, "client_id": 450}'

In [111]:
json_doc=[json.loads(doc_value)]

In [112]:
json_doc

[{'_id': {'$oid': '667bca5365714e7849d07a2d'},
  'comments': 'Followup Added',
  'email_reminder': 1,
  'email_reminder_date': {'$date': 1719408602000},
  'likely_purchase_date': {'$date': 1719964800000},
  'delete_event_data': 1,
  'auto': 0,
  'status': '1668585610133',
  'next_followup_mode': 'Call',
  'next_followup_mode_id': '1668406378088',
  'followup_owner_id': 'LQSDEALER1-007007',
  'followup_owner_name': 'GP User',
  'lead_status': 'Hot',
  'customer_email': 'akash.singhavi@adglobal360.com',
  'outlet_id': 'LQSDEALER1',
  'customer_name': 'Akash',
  'lead_owner_id': 'LQSDEALER1-007007',
  'lead_lob': 1,
  'preferred_date_time': '2024-06-26T13:30:02.000Z',
  'created_at': {'$date': 1719408555000},
  'lead_id': {'$oid': '667bca51b6292c3909ee4e46'},
  'followup_added_by_distributor': 1,
  'client_id': 450}]

In [118]:
# Convert data to Spark DataFrame
# Here, we convert each dictionary to a Row object for DataFrame creation
rows = [Row(**row) for row in json_doc]
doc_df = spark.createDataFrame(rows)

# Show DataFrame
doc_df.show(truncate=False)

+----------------------------------+--------------+--------------+------------------------+------------------------+-----------------+----+-------------+------------------+---------------------+-----------------+-------------------+-----------+------------------------------+----------+-------------+-----------------+--------+------------------------+------------------------+----------------------------------+-----------------------------+---------+
|_id                               |comments      |email_reminder|email_reminder_date     |likely_purchase_date    |delete_event_data|auto|status       |next_followup_mode|next_followup_mode_id|followup_owner_id|followup_owner_name|lead_status|customer_email                |outlet_id |customer_name|lead_owner_id    |lead_lob|preferred_date_time     |created_at              |lead_id                           |followup_added_by_distributor|client_id|
+----------------------------------+--------------+--------------+------------------------+---

In [115]:
doc_df.printSchema()

root
 |-- _id: map (nullable = true)
 |    |-- key: string
 |    |-- value: string (valueContainsNull = true)
 |-- comments: string (nullable = true)
 |-- email_reminder: long (nullable = true)
 |-- email_reminder_date: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- likely_purchase_date: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)
 |-- delete_event_data: long (nullable = true)
 |-- auto: long (nullable = true)
 |-- status: string (nullable = true)
 |-- next_followup_mode: string (nullable = true)
 |-- next_followup_mode_id: string (nullable = true)
 |-- followup_owner_id: string (nullable = true)
 |-- followup_owner_name: string (nullable = true)
 |-- lead_status: string (nullable = true)
 |-- customer_email: string (nullable = true)
 |-- outlet_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- lead_owner_id: string (nullable = true)
 |-- lead_lob: long (null

In [130]:
df=spark.read.format("text").option("header",True).load(path="/user/sovik/nag_issues/sample_lqs_26062024_v2.csv")
df_list=df.collect()
df_header=df_list[0].value
df_value=df_list[1].value
df_header_v1=df_header.split(",")
#header name
df_header_doc=df_header_v1[-1]
df_header_doc=df_header_doc[1:]
#header name
df_header_doc=df_header_doc[:-1]

#doc value
startswith=df_value.find('{""_id""')

doc_value=df_value[startswith:]

doc_value=doc_value.replace('""','"')
doc_value=doc_value[:-1]



In [131]:
doc_value

'{"_id": {"$oid": "65700192dd505706d3d62510"}, "lead_id": {"$oid": "65700125dd50572c6ed624f7"}, "outlet_id": "LQSDEALER1", "first_name": "Rfr", "middle_name": "Tty", "last_name": "Cgh", "phone": "7009685249", "customer_type": "Corporate", "customer_type_id": "1669198541604", "model": "JIMNY", "model_code": "JMN", "variant_code": "03", "variant": "ZETA", "color_code": "GG", "color": "GRANDEUR GREY", "vehicle_sale_type_id": "2", "vehicle_sale_type": "Institution", "currency_id": "1", "currency_name": "HUF", "base_price": "0", "payment_at_dealership": true, "pre_booking_currency_name": "HUF", "pre_booking_amount": "522", "pre_booking_currency_id": "1", "pre_booking_payment_mode": "Demand Draft", "pre_booking_payment_mode_id": "1674541067635", "payment_currency_name": "HUF", "payment_currency_id": "1", "payment_mode": "Cash + Finance", "payment_mode_id": "101", "refrence_number": "280", "financer_name": "TESTPARTY", "financer_id": "23232323", "finance_amount": "Yyy", "tentative_delivery_da

In [132]:
json_doc=[json.loads(doc_value)]

# Convert data to Spark DataFrame
# Here, we convert each dictionary to a Row object for DataFrame creation
rows = [Row(**row) for row in json_doc]
doc_df = spark.createDataFrame(rows)

# Show DataFrame
doc_df.show(truncate=False)

+----------------------------------+----------------------------------+----------+----------+-----------+---------+----------+-------------+----------------+-----+----------+------------+-------+----------+-------------+--------------------+-----------------+-----------+-------------+----------+---------------------+-------------------------+------------------+-----------------------+------------------------+---------------------------+---------------------+-------------------+--------------+---------------+---------------+-------------+-----------+--------------+------------------------+------------------------------------------------------------------------------------------+----------------+-------------------+--------------+----------------------+-------------------+--------------+------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [133]:
doc_df.select(['lead_id.$oid']).show()

+--------------------+
|                $oid|
+--------------------+
|65700125dd50572c6...|
+--------------------+

